In [2]:
import pandas as pd
import numpy as np

# Feature Selection

### Load in the data

In [5]:
data = pd.read_csv('dataset/spotify_songs_restructured_cleaned.csv')
data.head()

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13 00:00:00,pop,0.726,0.815,11.0,-4.969,0.03730,0.357000,0.693,120.757315,162600.0,Medium Popularity,Fast,Loud,December
1,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19 00:00:00,pop,0.718,0.930,7.0,-3.778,0.10200,0.204000,0.277,121.956000,169093.0,Medium Popularity,Fast,Loud,July
2,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05 00:00:00,pop,0.650,0.833,1.0,-4.672,0.03590,0.191022,0.725,123.976000,189052.0,Medium Popularity,Fast,Loud,March
3,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26 00:00:00,pop,0.449,0.856,5.0,-4.788,0.10645,0.176000,0.152,112.648000,187675.0,Medium Popularity,Moderate,Loud,July
4,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29 00:00:00,pop,0.542,0.903,4.0,-2.419,0.04340,0.191022,0.367,127.936000,207619.0,Medium Popularity,Fast,Loud,August


In [7]:
data.isnull().sum()

track_name                     0
track_artist                   0
track_popularity               0
track_album_name               0
track_album_release_date       0
playlist_genre                 0
danceability                   0
energy                         0
key                            0
loudness                       0
speechiness                    0
liveness                       0
valence                        0
tempo                          0
duration_ms                    0
popularity_category         1572
tempo_category                 3
loudness_category              0
release_month               1188
dtype: int64

In [6]:
from sklearn.feature_selection import f_classif, SelectKBest

X = data[['danceability', 'energy', 'tempo', 'loudness']]  # Feature set
y = data['popularity_category']  # Target

# # Apply the F-test
# selector = SelectKBest(f_classif, k='all')
# X_new = selector.fit_transform(X, y)
# 
# # Get the scores
# f_scores = selector.scores_
# f_scores

ValueError: Input contains NaN